In [153]:
import numpy as np
import pandas as pd
import altair as alt

In [154]:
stinson2019_df = pd.read_pickle("data/stinson2019/stinson2019_df")

In [155]:
stinson2019_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9929 entries, 0 to 9928
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   site            9929 non-null   object        
 1   date            9929 non-null   datetime64[ns]
 2   tree            9929 non-null   object        
 3   species         9929 non-null   object        
 4   tap             9929 non-null   object        
 5   sugar           8139 non-null   float64       
 6   sap_wt          9296 non-null   float64       
 7   year            9929 non-null   datetime64[ns]
 8   site_id         9929 non-null   object        
 9   tree_id         9929 non-null   object        
 10  tree_record_id  9929 non-null   object        
 11  tap_record_id   9929 non-null   object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 931.0+ KB


In [156]:
stinson2019_df["doy"] = pd.DatetimeIndex(stinson2019_df["date"]).dayofyear
stinson2019_df["year_int"] = pd.DatetimeIndex(stinson2019_df["year"]).year
stinson2019_df["yearly_sap"] = np.nan
stinson2019_df["yearly_sugar"] = np.nan
stinson2019_df['tree_tap'] = stinson2019_df['tree']+ '_' + stinson2019_df['tap']
stinson2019_df = stinson2019_df.sort_values(by="date")

abs_measurement = 0

for year in stinson2019_df["year_int"].unique():
    year_df = stinson2019_df[stinson2019_df["year_int"] == year]

    for site in stinson2019_df[stinson2019_df["year_int"] == year]["site"].unique():
        for tree in stinson2019_df[
            (stinson2019_df["year_int"] == year) & (stinson2019_df["site"] == site)
        ]["tree"].unique():
            for tap in stinson2019_df[
                (stinson2019_df["year_int"] == year)
                & (stinson2019_df["site"] == site)
                & (stinson2019_df["tree"] == tree)
            ]["tap"].unique():
                tap_yr_index = stinson2019_df.index[
                    (stinson2019_df["year_int"] == year)
                    & (stinson2019_df["site"] == site)
                    & (stinson2019_df["tree"] == tree)
                    & (stinson2019_df["tap"] == tap)
                ].tolist()
                yearly_sap = 0
                yearly_sugar = 0
                for measurement in tap_yr_index:
                    yearly_sap = np.nansum(
                        [yearly_sap, stinson2019_df.loc[measurement, "sap_wt"]]
                    )
                    yearly_sugar = np.nansum(
                        [
                            yearly_sugar,
                            stinson2019_df.loc[measurement, "sap_wt"]
                            * stinson2019_df.loc[measurement, "sugar"],
                        ]
                    )
                    stinson2019_df.loc[measurement, "yearly_sap"] = yearly_sap
                    stinson2019_df.loc[measurement, "yearly_sugar"] = yearly_sugar


#     yearly_sap = 0
#     yearly_sugar = 0
#     for measurement in list(range(len(year_df))):
#         yearly_sap+=stinson2019_df.iloc[abs_measurement,6]
#         yearly_sugar+=stinson2019_df.iloc[abs_measurement,6]*stinson2019_df.iloc[abs_measurement,5]
#         stinson2019_df.iloc[abs_measurement,13]=yearly_sap
#         stinson2019_df.iloc[abs_measurement,14]=yearly_sugar
#         abs_measurement+=1

In [223]:
a = alt.Chart(
    stinson2019_df[
        (stinson2019_df["tree"] == "HF1")
        & (stinson2019_df["year_int"] == 2012)
    ]
).mark_line().encode(
    y="yearly_sap",
    x="doy",
    color=alt.Color("tap", scale=alt.Scale(scheme="category10")),
)

In [232]:
b = alt.Chart(
    stinson2019_df[
        (stinson2019_df["tree"] == "HF1")
        & (stinson2019_df["year_int"] == 2012)
    ]
).mark_line(color='black').encode(
    y="yearly_sap_agg:Q",
    x="doy",
).transform_aggregate(
    yearly_sap_agg='sum(yearly_sap)',
    groupby=['doy']
)

In [233]:
a+b

alt.LayerChart(...)

In [210]:
stinson2019_df[
    (stinson2019_df["tree"] == "HF1")
    & (stinson2019_df["year_int"] == 2012)
    & ((stinson2019_df["tap"] == "A") | (stinson2019_df["tap"] == "B"))]

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,doy,year_int,yearly_sap,yearly_sugar
0,Harvard Forest,2012-02-16,HF1,ACSA,A,2.7,4.24,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-16,HF.ACSA.HF1.A.2012-02-16,47,2012,4.24,11.448
1,Harvard Forest,2012-02-16,HF1,ACSA,B,2.4,1.08,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-16,HF.ACSA.HF1.B.2012-02-16,47,2012,1.08,2.592
24,Harvard Forest,2012-02-17,HF1,ACSA,B,2.3,3.96,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-17,HF.ACSA.HF1.B.2012-02-17,48,2012,5.04,11.700
23,Harvard Forest,2012-02-17,HF1,ACSA,A,2.6,6.47,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-17,HF.ACSA.HF1.A.2012-02-17,48,2012,10.71,28.270
47,Harvard Forest,2012-02-18,HF1,ACSA,B,2.4,3.54,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-18,HF.ACSA.HF1.B.2012-02-18,49,2012,8.58,20.196
46,Harvard Forest,2012-02-18,HF1,ACSA,A,2.6,4.22,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-18,HF.ACSA.HF1.A.2012-02-18,49,2012,14.93,39.242
70,Harvard Forest,2012-02-19,HF1,ACSA,B,2.3,3.41,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-19,HF.ACSA.HF1.B.2012-02-19,50,2012,11.99,28.039
69,Harvard Forest,2012-02-19,HF1,ACSA,A,2.6,4.50,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-19,HF.ACSA.HF1.A.2012-02-19,50,2012,19.43,50.942
93,Harvard Forest,2012-02-21,HF1,ACSA,B,2.5,1.14,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-21,HF.ACSA.HF1.B.2012-02-21,52,2012,13.13,30.889
92,Harvard Forest,2012-02-21,HF1,ACSA,A,2.3,3.50,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-21,HF.ACSA.HF1.A.2012-02-21,52,2012,22.93,58.992


In [244]:
alt.Chart(
    stinson2019_df[(stinson2019_df["year_int"] == 2012)
    ]
).mark_line(color='black').encode(
    y="yearly_sap_agg:Q",
    x="doy",
    color='tree'
).transform_aggregate(
    yearly_sap_agg='sum(yearly_sap)',
    groupby=['doy','tree']
)

alt.Chart(...)

In [104]:
stinson2019_df['year_int'].value_counts()

2016    2853
2017    2368
2014    1614
2015    1587
2013    1116
2012     391
Name: year_int, dtype: int64

In [106]:
stinson2019_df[stinson2019_df['year_int']==2012]['doy'].unique()

array([47, 48, 49, 50, 52, 53, 54, 58, 59, 67, 68, 69, 71, 72, 73, 74, 79])

In [157]:
stinson2019_df[(stinson2019_df["tree"] == "HF1") & (stinson2019_df["year_int"] == 2012) & (stinson2019_df['tap']=='A')]

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,doy,year_int,yearly_sap,yearly_sugar
0,Harvard Forest,2012-02-16,HF1,ACSA,A,2.7,4.24,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-16,HF.ACSA.HF1.A.2012-02-16,47,2012,4.24,11.448
23,Harvard Forest,2012-02-17,HF1,ACSA,A,2.6,6.47,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-17,HF.ACSA.HF1.A.2012-02-17,48,2012,10.71,28.270
46,Harvard Forest,2012-02-18,HF1,ACSA,A,2.6,4.22,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-18,HF.ACSA.HF1.A.2012-02-18,49,2012,14.93,39.242
69,Harvard Forest,2012-02-19,HF1,ACSA,A,2.6,4.50,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-19,HF.ACSA.HF1.A.2012-02-19,50,2012,19.43,50.942
92,Harvard Forest,2012-02-21,HF1,ACSA,A,2.3,3.50,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-21,HF.ACSA.HF1.A.2012-02-21,52,2012,22.93,58.992
115,Harvard Forest,2012-02-22,HF1,ACSA,A,2.6,7.18,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-22,HF.ACSA.HF1.A.2012-02-22,53,2012,30.11,77.660
138,Harvard Forest,2012-02-23,HF1,ACSA,A,2.5,4.94,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-23,HF.ACSA.HF1.A.2012-02-23,54,2012,35.05,90.010
161,Harvard Forest,2012-02-27,HF1,ACSA,A,2.6,4.62,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-27,HF.ACSA.HF1.A.2012-02-27,58,2012,39.67,102.022
184,Harvard Forest,2012-02-28,HF1,ACSA,A,2.5,8.40,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-28,HF.ACSA.HF1.A.2012-02-28,59,2012,48.07,123.022
207,Harvard Forest,2012-03-07,HF1,ACSA,A,2.2,4.82,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-03-07,HF.ACSA.HF1.A.2012-03-07,67,2012,52.89,133.626
